In [9]:
!pip install -U langchain langchain-community sentence-transformers chromadb

In [10]:
#loading of the given documents
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader


#Chris Olah's blog using a web scraper
olah_loader = WebBaseLoader("https://colah.github.io/posts/2015-08-Understanding-LSTMs/")
olah_docs = olah_loader.load()

# CMU LSTM PDF manually in Colab
from google.colab import files
uploaded = files.upload()  # upload LSTM.pdf file here , an option pops up to cho0se from local files

# Load PDF
pdf_loader = PyPDFLoader("LSTM.pdf")  # Use the uploaded file
cmu_docs = pdf_loader.load()

# Combine both document sources
all_docs = olah_docs + cmu_docs


Saving LSTM.pdf to LSTM (2).pdf


In [11]:
#splitting text into smaller chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.split_documents(all_docs)
#we are splitting the documents to chunks of 200 each for optimized use .


Embeddings: Numeric vectors that represent the meaning of text.

HuggingFace : librarry for building, sharing, and deploying machine learning models

In [13]:
#creating free version text embeddings through huggingface
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Create vector embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Store in Chroma vector database
vector_db = Chroma.from_documents(docs, embeddings, persist_directory="./rag_db")





chroma : A local vector database to store and retrieve documents by similarity


In [14]:
from transformers import pipeline

# Loading  a lightweight text generation model .
# Model used here "flan-t5-base"
rag_pipeline = pipeline("text2text-generation", model="google/flan-t5-small")


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


retriever :Searches document database for relevant chunks

In [15]:
#creating a retrievel QA  chain

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

# Wrap the pipeline into a LangChain LLM interface
local_llm = HuggingFacePipeline(pipeline=rag_pipeline)

# Created the retrieval-based QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever()
)
# A normal llm like gpt-4 doesnt know specific documents and tries to guess based on pretraining ,retrievalQA function  chain
#indexes the documents ,convenrts them into embeddings and retrieves nost relevant part .

<ipython-input-15-be9013578e1a>:6: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  local_llm = HuggingFacePipeline(pipeline=rag_pipeline)


In [17]:
from langchain.chains import RetrievalQAWithSourcesChain

qa_chain_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever()
)

query = "What are input and output gates in LSTMs?"
result = qa_chain_with_sources(query)
print("Answer:", result['answer'])

# more questions can be asked egs:
#what is the role of forget gate in LSTM ?
#how is lstm different than a vanilla RNN ?



Answer: LSTMs are a way to optionally let information through. They are composed out of a sigmoid neural net layer and a pointwise multiplication operation.


 Models used and reasoning
 1. Embedding Model- for converting text into vector form:
Model used: all-MiniLM-L6-v2
From: Hugging Face (sentence-transformers family)
Purpose: Converts the documents -Chris Olah's blog and CMU notes into numerical vectors so that the system can semantically search for relevant parts based on a user's query.

This is a lightweight, fast, and widely used model suitable for semantic search and RAG tasks.

 2. Language Model -for generating answers based on retrieved content:
Model used: flan-t5-base
From: Hugging Face -Google's FLAN-T5 series.
Purpose: This model takes the user query and the relevant context retrieved using embeddings and generates a natural-language answer.

I  initially tried HuggingFaceHub, which required an API token. Later, I  switched to loading FLAN-T5 locally (without needing tokens), so everything worked freely in Colab.